<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/iDDY_Dataset_Exploratory_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 15qdMBG9IDtLlIc4WTHbwirvjot4ggtyF   # Google Drive file ID

Downloading...
From: https://drive.google.com/uc?id=15qdMBG9IDtLlIc4WTHbwirvjot4ggtyF
To: /content/threadPost15Column.tsv
50.1MB [00:00, 158MB/s] 


In [ ]:
import pandas as pd
from tqdm import tqdm

# Using readlines()
FILENAME='threadPost15Column.tsv'

file1 = open(FILENAME, 'r')
Lines = file1.readlines()
 
count = 0

tid = []
timestamp = []
subject = []
msg_01 = []
msg_02 = []
msg_03 = []
msg_04 = []
msg_05 = []
msg_06 = []
label_01 = []
label_02 = []
label_03 = []
label_04 = []
label_05 = []
label_06 = []

# Strips the newline character
for line in tqdm(Lines):
  count += 1
  cols = line.split("\t\t")
  if (len(cols) != 15):
    print("Line{}: cols={} {}".format(count, len(cols), line.strip()))
  else:
    # exclude message-less & label-less threads
    if cols[9] == 'neu' or cols[9] == 'pos' or cols[9] == 'neg':   
      tid.append(cols[0])
      timestamp.append(cols[1])
      subject.append(cols[2])
      msg_01.append(cols[3])
      msg_02.append(cols[4])
      msg_03.append(cols[5])
      msg_04.append(cols[6])
      msg_05.append(cols[7])
      msg_06.append(cols[8])
      label_01.append(cols[9])
      label_02.append(cols[10])
      label_03.append(cols[11])
      label_04.append(cols[12])
      label_05.append(cols[13])
      label_06.append(cols[14].strip())   # remove \n

print('Read {}, {} lines'.format(FILENAME,count))

100%|██████████| 215845/215845 [00:00<00:00, 459669.27it/s]

Read threadPost15Column.tsv, 215845 lines


Number of threads (excluded **message-less**, **label-less** threads)

In [ ]:
# Create a dataframe from filtered raw text
df = pd.DataFrame({
    'tid': tid,
    'timestamp': timestamp,
    'subject': subject,
    'msg_01': msg_01,
    'msg_02': msg_02,
    'msg_03': msg_03,
    'msg_04': msg_04,
    'msg_05': msg_05,
    'msg_06': msg_06,
    'label_01': label_01,
    'label_02': label_02,
    'label_03': label_03,
    'label_04': label_04,
    'label_05': label_05,
    'label_06': label_06
})
print('dataframe: {} rows, {} columns'.format(len(df),len(df.columns)))

dataframe: 58194 rows, 15 columns


In [ ]:
df.head(10)

,tid,timestamp,subject,msg_01,msg_02,msg_03,msg_04,msg_05,msg_06,label_01,label_02,label_03,label_04,label_05,label_06
0,DISCU_57_29839462,1619013446,華為無人駕駛系統再帶領A股進入大牛市,smile_o05.gif,cow,buy,升,,,neu,,,,,
1,DISCU_57_29839355,1619010602,夜期已經開始.........,"下跌乏力,明天大爆",天日284二八八兩邊發 收八六,<img src=,,,,neu,neu,neu,,,
2,DISCU_57_29839333,1619009841,跌市已經完結...........…,"終於安心晒,跌500點果陣已入牛再出發",聽日又跌,即係牛安心上路?🤔,好快五窮月,,,neu,neu,neu,,,
3,DISCU_57_29838788,1618993010,京東9618 vs 美團3690,現價: 京東 $287.20(PE 14.74) 美團$282.20(PE 292.30) ...,3690,老千股,,,,neu,neu,neu,,,
4,DISCU_57_29838728,1618991023,美團(3690)被大戶輪流洗倉了,美團(3690)被大戶輪流洗倉了,,,,,,neu,,,,,
5,DISCU_57_29838700,1618990508,588.5騰訊！快D！！,等左好耐。,<img src=,,,,,neu,neu,,,,
6,DISCU_57_29838693,1618990301,大市缺乏經濟火車頭 印度SENSEX接大陸棒？,大陸股到哂頂啦 即是香港股到哂頂啦 整固期可以長達一年以上 - 買過CSI300...,,,,,,neu,,,,,
7,DISCU_57_29838625,1618987645,快手！ 歷史低啦！！ 快手啦！！！🎉🎉🎉,你今日快手d 啦 20210421_ea03f3dc74f222fd675fWcG8h9Sp...,招股價幾多 cry.gif cry.gif,115 smile_35.gif,韭菜1斤斤的抬上貨車🚚運走🤤,老千 cry.gif cry.gif,抖音上。佢就係咁先,neu,neu,neu,neu,neu,neu
8,DISCU_57_29838618,1618987319,【滙豐HSBC】滙豐多名倫敦高層改駐香港傳惹內部不滿 消息：揣測無事實根據,【滙豐HSBC】滙豐多名倫敦高層改駐香港傳惹內部不滿 消息:揣測無事實根據,總部幾時搬番黎先 smile_34.gif smile_34.gif smile_34....,這己表示香港市場比英國重要。,來自香港同亞洲利潤遠大於英國歐洲,,,neu,neu,neu,neu,,
9,DISCU_57_29838599,1618986286,調查指小米全球智能手機市佔率排第三 銷量增八成,,勿買。。。遲早好似華為咁被搞 smile_42.gif,,,,,neu,neu,,,,


In [ ]:
df.loc[df['tid'] == 'DISCU_57_29837114']

,tid,timestamp,subject,msg_01,msg_02,msg_03,msg_04,msg_05,msg_06,label_01,label_02,label_03,label_04,label_05,label_06
52,DISCU_57_29837114,1618908481,9923移卡暴升10%,女股神愛股啟動 幾時到 閱文 眾安 平安好醫生 ????,請問9923睇幾多?,funk.gif funk.gif 又爆升,高位120蚊跌落黎 怕咩 吉利36蚊跌落黎 依家起緊 小米又係35蚊跌落黎 又起緊 ...,<img src=,今日又升5% .話咁快71蚊啦,neu,neu,neu,neu,neu,


In [ ]:
df.loc[df['tid'] == 'DISCU_57_29756443']

,tid,timestamp,subject,msg_01,msg_02,msg_03,msg_04,msg_05,msg_06,label_01,label_02,label_03,label_04,label_05,label_06
2379,DISCU_57_29756443,1615039463,星期五美股咁樣，咁星期一既港股如無意外.....,升完會否跌呢??,美股跌咗100點?,美國細價股洗倉,,,,neu,neu,neu,,,


Number of 'neu', 'pos' and 'neg' labels?

In [ ]:
columns = ['label_01', 'label_02', 'label_03', 'label_04', 'label_05', 'label_06']
neu = 0
pos = 0
neg = 0

for col in columns:
  cneu = df[col].str.count('neu').sum()
  neu += cneu
  cpos = df[col].str.count('pos').sum()
  pos += cpos
  cneg = df[col].str.count('neg').sum()
  neg += cneg
  print('{}, {}, neu: {}, pos: {}, neg: {}'.format(col,pd.unique(df[col]),cneu,cpos,cneg))

print()
print('neu: {}, pos: {}, neg: {}'.format(neu, pos, neg))

label_01, ['neu' 'pos' 'neg'], neu: 44589, pos: 8003, neg: 5602
label_02, ['' 'neu' 'neg' 'pos'], neu: 37952, pos: 5693, neg: 5426
label_03, ['' 'neu' 'pos' 'neg'], neu: 33560, pos: 4908, neg: 4809
label_04, ['' 'neu' 'neg' 'pos'], neu: 30007, pos: 4367, neg: 4094
label_05, ['' 'neu' 'neg' 'pos'], neu: 26959, pos: 3964, neg: 3717
label_06, ['' 'neu' 'neg' 'pos'], neu: 24463, pos: 3484, neg: 3364

neu: 197530, pos: 30419, neg: 27012


How many messages are there per thread (subject)?



In [ ]:
columns = ['label_01', 'label_02', 'label_03', 'label_04', 'label_05', 'label_06']
msg = 0
for index, row in df.iterrows():
  msg_per_subject = 0
  for col in columns:
    if row[col] == 'neu' or row[col] == 'pos' or row[col] == 'neg':
      msg_per_subject += 1
  msg += msg_per_subject

print('{:.1f}'.format(msg/len(df)))

4.4


Average message length

In [ ]:
columns = ['subject', 'msg_01', 'msg_02', 'msg_03', 'msg_04', 'msg_05', 'msg_06']

mean_msg_length = []
for col in columns:
  mean_msg_length.append(df[col].apply(len).mean())

# print(mean_msg_length)

for col, avg_len in zip(columns,mean_msg_length):
  print('{}: {:.1f}'.format(col,avg_len))

subject: 18.3
msg_01: 84.3
msg_02: 24.8
msg_03: 20.9
msg_04: 18.7
msg_05: 16.8
msg_06: 15.2


Find most frequent tokens, nouns, verbs, adjectives and named entities

In [ ]:
!pip install --upgrade --quiet spacy[cuda102]
!python -m spacy download zh_core_web_lg
!git clone https://github.com/raymondwcs/learning_spacy

     |████████████████████████████████| 60.7 MB 23 kB/s 
     |████████████████████████████████| 603.8 MB 8.5 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')
fatal: destination path 'learning_spacy' already exists and is not an empty directory.


In [ ]:
!nvidia-smi -L

GPU 0: Tesla P4 (UUID: GPU-8145bcf5-984b-ca20-a687-bd8c31e86d23)


In [ ]:
import spacy

tk = []       # tokens
noun = []     # nouns, proper nouns
verb = []     # verbs
adj = []      # adjectives
ne = []       # named entities

# print(spacy.prefer_gpu()) # or spacy.require_gpu()
print(spacy.require_gpu())
nlp = spacy.load('zh_core_web_lg')
# Use trained pkuseg model on Traditional Chinese text
# Download folder "models" from:
# https://github.com/raymondwcs/learning_spacy/tree/main/spacy_pkuseg
nlp.tokenizer.initialize(pkuseg_model='./learning_spacy/spacy_pkuseg/models')

from spacy.lang.zh.stop_words import STOP_WORDS
spacy_stopwords = STOP_WORDS
for w in spacy_stopwords:
  nlp.vocab[w].is_stop = True

columns = ['subject','msg_01','msg_02','msg_03','msg_04','msg_05','msg_06']
for index, row in tqdm(df.iterrows(), leave=True):
  for col in columns:
    msg = row[col].strip()
    if (len(msg) == 0):
      continue
    doc = nlp(msg)
    # remove stop words
    tokens = [token for token in doc if not token.is_stop]
    # for token in doc:
    for token in tokens:
        tk.append(token.text)
        if token.pos_ == 'NOUN' or token.pos_ == 'PROPN':
            noun.append(token.text)
        elif token.pos_ == 'VERB':
            verb.append(token.text)
        elif token.pos_ == 'ADJ':
            adj.append(token.text)
    for ent in doc.ents:
        ne.append('(%s, %s)' % (ent.text, ent.label_))

False


58194it [36:40, 26.44it/s]


In [ ]:
pd.set_option('display.max_rows', None) 

In [ ]:
pd.Series(tk).value_counts().head(3000)

                               145813
係                               26733
                                26251
                                22566
個                               22476
cry.gif                         22387
點                               19297
會                               19168
買                               18274
升                               18125
跌                               16446
元                               16426
smile_34.gif                    15754
港股                              12165
中國                              11828
                                11800
香港                              11692
幾                               10916
今日                              10776
公司                              10394
年                               10062
佢                                9467
「                                9362
過                                9189
將                                8870
騰訊                               8434
美國          

In [ ]:
pd.Series(noun).value_counts().head(3000)

cry.gif               22274
港股                    12165
中國                    11828
香港                    11692
今日                    10776
公司                    10394
美國                     8413
騰訊                     8197
smile_34.gif           7956
股市                     7930
市場                     7231
smile_22.gif           5795
股票                     5676
小米                     5556
smile_38.gif           5540
股價                     5268
月                      4993
錢                      4931
smile_30.gif           4729
美股                     4694
大戶                     4541
牛                      4466
指數                     4448
銀行                     4361
消息                     4275
業績                     4266
集團                     4148
夜期                     4079
經濟                     3929
業務                     3805
大市                     3494
恒指                     3480
lol.gif                3459
funk.gif               3421
匯豐                     3388
??                  

In [ ]:
pd.Series(adj).value_counts().head(3000)

新                                   4409
高                                   2086
恆指                                  1265
低                                    991
相關                                   836
內地                                   796
前                                    748
個                                    728
壞                                    703
未來                                   691
沽空                                   652
這個                                   606
有關                                   534
無                                    477
最低                                   474
賭變                                   428
幾多                                   422
大型                                   417
死                                    402
平均                                   369
smile_41.gif                         359
唯一                                   344
人工                                   328
副                                    318
整體              

In [ ]:
pd.Series(verb).value_counts().head(3000)

係                     24802
買                     17804
升                     17238
跌                     16361
smile_34.gif           7797
為                      7604
做                      6286
過                      5832
想                      5751
睇                      5410
指                      5327
無                      4264
走                      4132
入                      4122
見                      3857
賺                      3670
玩                      3630
smile_o03.gif          3586
會                      3457
講                      3400
smile_30.gif           3170
完                      3144
smile_41.gif           3115
上市                     3026
支持                     2947
爆升                     2891
增長                     2852
投資                     2780
高                      2763
smile_47.gif           2640
炒                      2599
出現                     2569
成                      2517
smile_o12.gif          2490
輸                      2447
smile_27.gif        

In [ ]:
pd.Series(ne).value_counts().head(3000)

(香港, GPE)                                             9710
(今日, DATE)                                            6821
(港股, GPE)                                             6808
(美國, GPE)                                             6574
(中國, GPE)                                             6073
(騰訊, ORG)                                             5664
(三, CARDINAL)                                         3620
(3, CARDINAL)                                         3328
(小米, ORG)                                             3135
(明天, DATE)                                            3065
(2, CARDINAL)                                         2987
(今年, DATE)                                            2971
(一, CARDINAL)                                         2907
(700, CARDINAL)                                       2819
(1, CARDINAL)                                         2749
(美股, GPE)                                             2645
(第一, ORDINAL)                                         22